In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    with open('dataset/english_german_translation.txt','r',encoding = 'utf-8') as text_file:
        lines = text_file.readlines()
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s).replace('.','').strip() for s in l.split('\t')][0:2] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    with open('dataset/english_german_translation.txt','r',encoding = 'utf-8') as text_file:
        lines = text_file.readlines()
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s).replace('.','').strip() for s in l.split('\t')][0:2] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 255817 sentence pairs
Trimmed to 14698 sentence pairs
Counting words...
Counted words:
fra 5504
eng 3480
['ich bringe tom hier raus', 'i m getting tom out of here']


<div class="alert alert-info"><p>There are other forms of attention that work around the length
  limitation by using a relative position approach</p></div>

## Training

### Preparing Training Data

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.

In [2]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

train_pairs = [tensorsFromPair(i) for i in pairs]

## The Seq2Seq Model

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A [Sequence to Sequence network](https://arxiv.org/abs/1409.3215)_, or
seq2seq network, or [Encoder Decoder
network](https://arxiv.org/pdf/1406.1078v3.pdf)_, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.


With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




### The Encoder

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [4]:
hidden_size = 512

## Encoder analysis - for single input 

In [5]:
class ExtEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,embedding_dim):
        super(ExtEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size)

    def forward(self, input, hidden):
        print(f'initial hidden shape : {hidden.shape}')
        print(f'input shape : {input.shape}')
        embedded = self.embedding(input)
        print(f'after embedding :{embedded.shape}')
        output = embedded
        print(output.shape,hidden.shape)
        output, hidden = self.gru(output, hidden)
        print(f'rnn output shape : {output.shape}')
        print(f'rnn hidden shape : {hidden.shape}')
        return output, hidden

    def initHidden(self):
        return torch.zeros(1,self.hidden_size, device=device)

In [6]:
single_encoder = ExtEncoderRNN(input_lang.n_words, hidden_size=5,embedding_dim = 4).to(device)

# single input 
test_input = train_pairs[0][0][0]
test_hidden = single_encoder.initHidden()
output,hidden = single_encoder(test_input,test_hidden)

initial hidden shape : torch.Size([1, 5])
input shape : torch.Size([1])
after embedding :torch.Size([1, 4])
torch.Size([1, 4]) torch.Size([1, 5])
rnn output shape : torch.Size([1, 5])
rnn hidden shape : torch.Size([1, 5])


### The Decoder

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




#### Simple Decoder

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).

In [7]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

#### Attention Decoder

If only the context vector is passed between the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.


Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

In [8]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [45]:
class ExtAttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,embedding_size,dropout_p=0.1, max_length=MAX_LENGTH):
        super(ExtAttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding_size = embedding_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.embedding = nn.Embedding(self.output_size,self.embedding_size)
        self.attn = nn.Linear(self.embedding_size+self.hidden_size, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size + self.embedding_size, self.hidden_size)
        
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        
        print(f'decoder input shape : {input.shape}')
        print(f'decoder hidden shape : {hidden.shape}')
        print(f'encoder outputs shape : {encoder_outputs.shape}')
        
        embedded = self.embedding(input)
        print(f'input after emedding : {embedded.shape}')
        embedded = self.dropout(embedded)
        print(f'embedded[0] shape : {embedded[0].shape}')
        print(torch.cat((embedded[0],hidden),1).shape)
        print(self.attn(torch.cat((embedded[0], hidden), 1)).shape)
        print(embedded[0].shape,hidden.shape)
        
        print(torch.cat((embedded[0], hidden), 1).shape)
        
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden), 1)), dim=1)
        print(attn_weights)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))
        print(attn_applied)
        print(f'attn applied shape : {attn_applied.shape}')
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        print(output)
        print(f'output shape : {output.shape}')
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        output = output.squeeze(1)
        output, hidden = self.gru(output, hidden)
        print(output.shape,hidden.shape)
        output = F.log_softmax(self.out(output), dim=1)
        print(output.shape)
        
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [48]:
single_att_decoder = ExtAttnDecoderRNN(hidden_size, output_lang.n_words,100,dropout_p=0.1).to(device)

In [49]:
single_att_decoder(decoder_input,decoder_hidden,encoder_outputs)

decoder input shape : torch.Size([1, 1])
decoder hidden shape : torch.Size([1, 32])
encoder outputs shape : torch.Size([10, 32])
input after emedding : torch.Size([1, 1, 100])
embedded[0] shape : torch.Size([1, 100])
torch.Size([1, 132])
torch.Size([1, 10])
torch.Size([1, 100]) torch.Size([1, 32])
torch.Size([1, 132])
tensor([[0.1310, 0.0733, 0.0468, 0.1005, 0.0996, 0.0982, 0.1171, 0.1057, 0.1188,
         0.1091]], grad_fn=<SoftmaxBackward0>)
tensor([[[ 0.2166, -0.1217,  0.1057,  0.1919, -0.0188, -0.0062, -0.1873,
           0.1820, -0.2013,  0.1502,  0.0612, -0.0735, -0.0885, -0.0549,
          -0.2370, -0.0243, -0.1164, -0.0695, -0.0261, -0.1380, -0.1049,
           0.2002,  0.1645,  0.0958, -0.1100,  0.0036, -0.0669,  0.0629,
          -0.1162, -0.0937, -0.0352,  0.0223]]], grad_fn=<BmmBackward0>)
attn applied shape : torch.Size([1, 1, 32])
tensor([[ 1.3108,  0.1656, -0.4166,  0.6887, -0.3211,  0.3964,  1.5861, -0.7702,
          1.1397, -0.2677,  0.5646,  1.2591, -0.1190, -0.4722,

(tensor([[-8.1592, -8.0983, -8.0075,  ..., -7.9301, -7.8400, -7.8108]],
        grad_fn=<LogSoftmaxBackward0>),
 tensor([[-0.0565, -0.3942,  0.0849, -0.0896, -0.3273, -0.2901, -0.2603,  0.2940,
          -0.2104,  0.2771, -0.1109, -0.0987, -0.2969, -0.4606, -0.5245, -0.2944,
          -0.4583, -0.3871, -0.2190, -0.1908, -0.3645,  0.1680,  0.0607,  0.3192,
           0.1152,  0.1412, -0.3229,  0.3192, -0.2965, -0.5381, -0.4760, -0.0215]],
        grad_fn=<SqueezeBackward1>),
 tensor([[0.1310, 0.0733, 0.0468, 0.1005, 0.0996, 0.0982, 0.1171, 0.1057, 0.1188,
          0.1091]], grad_fn=<SoftmaxBackward0>))

### Training the Model

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but [when the trained
network is exploited, it may exhibit
instability](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf)_.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [22]:
class ExtEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,embedding_dim):
        super(ExtEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size)

    def forward(self, input, hidden):
        print(f'initial hidden shape : {hidden.shape}')
        print(f'input shape : {input.shape}')
        embedded = self.embedding(input)
        print(f'after embedding :{embedded.shape}')
        output = embedded
        output, hidden = self.gru(output, hidden)
        print(f'rnn output shape : {output.shape}')
        print(f'rnn hidden shape : {hidden.shape}')
        return output, hidden

    def initHidden(self):
        return torch.zeros(1,self.hidden_size, device=device)

In [37]:
hidden_size = 32
embedding_dim = 100
single_encoder = ExtEncoderRNN(input_lang.n_words, hidden_size,embedding_dim).to(device)
# single input 
input_tensor = train_pairs[0][0]
target_tensor = train_pairs[0][1]
input_length = input_tensor.size(0)
target_length = target_tensor.size(0)

encoder_hidden = single_encoder.initHidden()
encoder_outputs = torch.zeros(10,single_encoder.hidden_size)
for ei in range(input_length): 
    encoder_output,encoder_hidden = single_encoder(input_tensor[ei],encoder_hidden)
    encoder_outputs[ei] = encoder_output[0]
    print('---------------------------------------')
decoder_input = torch.tensor([[SOS_token]], device=device)
decoder_hidden = encoder_hidden

initial hidden shape : torch.Size([1, 32])
input shape : torch.Size([1])
after embedding :torch.Size([1, 100])
rnn output shape : torch.Size([1, 32])
rnn hidden shape : torch.Size([1, 32])
---------------------------------------
initial hidden shape : torch.Size([1, 32])
input shape : torch.Size([1])
after embedding :torch.Size([1, 100])
rnn output shape : torch.Size([1, 32])
rnn hidden shape : torch.Size([1, 32])
---------------------------------------
initial hidden shape : torch.Size([1, 32])
input shape : torch.Size([1])
after embedding :torch.Size([1, 100])
rnn output shape : torch.Size([1, 32])
rnn hidden shape : torch.Size([1, 32])
---------------------------------------
initial hidden shape : torch.Size([1, 32])
input shape : torch.Size([1])
after embedding :torch.Size([1, 100])
rnn output shape : torch.Size([1, 32])
rnn hidden shape : torch.Size([1, 32])
---------------------------------------
initial hidden shape : torch.Size([1, 32])
input shape : torch.Size([1])
after embed

In [24]:
encoder_outputs.shape

torch.Size([10, 32])

In [25]:
encoder_outputs

tensor([[-0.2226,  0.2231, -0.1408, -0.3557,  0.0480, -0.2754, -0.1883,  0.0357,
          0.0505, -0.0541, -0.0434,  0.1005, -0.3365, -0.2832,  0.3067,  0.4034,
          0.5261, -0.6597,  0.5050,  0.0251, -0.2940,  0.4238,  0.1882,  0.0084,
         -0.4640,  0.0490, -0.2018, -0.0625,  0.1574,  0.6681, -0.5048, -0.4688],
        [-0.3161,  0.0760,  0.0143, -0.6007, -0.3308,  0.0729, -0.4699, -0.1163,
          0.2389, -0.2370,  0.1256, -0.4235, -0.4506, -0.0046, -0.2461,  0.0774,
          0.0867,  0.1212,  0.5752,  0.3531, -0.1975,  0.4425,  0.2415,  0.2779,
         -0.4265,  0.1083,  0.1686, -0.2448,  0.0586,  0.4428, -0.7762,  0.3512],
        [-0.6382, -0.4839,  0.1170, -0.2116, -0.4630,  0.1612, -0.0673,  0.2856,
         -0.4797, -0.3873, -0.6783,  0.1106, -0.1549, -0.0153, -0.3844,  0.0085,
          0.7429, -0.0855,  0.1363,  0.3501, -0.0947,  0.2132,  0.2027,  0.1221,
         -0.0153,  0.1697, -0.4586, -0.3450, -0.3735,  0.4819, -0.6788, -0.0717],
        [-0.4334,  0.4492

In [26]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [33]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [27]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

### Plotting results

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [28]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Evaluation

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [29]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [32]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Training and Evaluating

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [35]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
trainIters(encoder1, attn_decoder1, 10000, print_every=1000)

0m 0s (- 0m 0s) (1 100%) 8.1158


In [36]:
evaluateRandomly(encoder1, attn_decoder1)

> er ist sehr angstlich vor hunden
= he is very afraid of dogs
< jokes jokes bicycle workshop workshop workshop workshop brooch redhead loser

> ich bin allergisch gegen mais
= i m allergic to corn
< jokes sharp groggy eat workshop workshop existence workshop workshop nobody

> ich bin interessiert
= i m interested
< sharp sharp groggy existence thirties dime existence dime apologize bicycle

> ich suche meine stiefel
= i m looking for my boots
< plain nobody dime drugs saturday died jokes drugs workshop workshop

> ich bin zu allem zu mude
= i m too tired to do anything
< sharp groggy existence thirties dime existence millionaire view existence workshop

> ich offne das paket
= i m going to unwrap the package
< death short hear dime chop chop am chop chop am

> wir sind designerinnen
= we re designers
< sharp sharp thirties then lot cannibals dime drugs workshop biologist

> er ist ein verheirateter mann
= he s a married man
< monthly jokes bicycle certainly workshop workshop workshop

### Visualizing Attention

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run ``plt.matshow(attentions)`` to see attention output
displayed as a matrix, with the columns being input steps and rows being
output steps:




In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())

For a better viewing experience we will do the extra work of adding axes
and labels:




In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("ich bin beharrlich")